# FmKorea 하이닉스 커뮤니티 인기글 크롤링 코드

### 실행순서

1. 1번째 셀 실행 (크롤링을 위한 함수 정의)
2. 2번째 셀 실행 (검색 키워드 "하이닉스"로 크롤링 실행, df_hynix 데이터프레임 return)
3. 3번째 셀 실행 (검색 키워드 "하닉"으로 크롤링 실행, df_hanic 데이터프레임 return)
4. 4번째 셀 실행 (생성된 df 2개에서 중복되는 게시글 삭제 및 csv 파일생성)

In [4]:

import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import re

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/143.0.0.0 Safari/537.36"
    ),
    "Referer": "https://www.fmkorea.com/",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
}

# 날짜 문자열을 정규화(통일)하는 함수
# - "17:27" 처럼 시간만 있으면 오늘 날짜(YYYY-MM-DD)로 치환
# - "2026.01.12" 처럼 점(.)으로 된 날짜는 "-"로 변경
def normalize_date(date_str: str) -> str:
    today = datetime.now().strftime("%Y-%m-%d")
    s = (date_str or "").strip()
    if re.match(r"^\d{1,2}:\d{2}$", s):
        return today
    if "." in s:
        return s.replace(".", "-")
    return s

# 검색 결과 페이지를 순회하며 게시글 리스트를 수집해서 DataFrame으로 반환
# start_page ~ end_page: 수집할 페이지 범위
def crawl_one(session: requests.Session, url_base: str, source_name: str,
              start_page=1, end_page=10, sleep_sec=2) -> pd.DataFrame:
    data = {"탭": [], "제목": [], "글쓴이": [], "날짜": [], "조회": [], "추천": [],
            "post_url": [], "source": []}

    for page in range(start_page, end_page + 1):
        url = url_base.format(page)

        r = session.get(url, timeout=20)
        r.raise_for_status()

        soup = bs(r.text, "lxml")

        rows = soup.select("table.bd_lst.bd_tb_lst.bd_tb tbody tr")
        if not rows:
            print(f"[{source_name}] {page}페이지: rows=0 → 중단")
            break

        page_added = 0
        for tr in rows:
            cate_a = tr.select_one("td.cate span")
            title_a = tr.select_one("td.title a.hx")
            author_a = tr.select_one("td.author a")
            time_td = tr.select_one("td.time")
            mno_tds = tr.select("td.m_no")

            if not (cate_a and title_a and author_a and time_td and len(mno_tds) >= 2):
                continue

            views = mno_tds[0].text.strip()
            votes = mno_tds[1].text.strip()

            href = title_a.get("href", "")
            post_url = "https://www.fmkorea.com" + href if href.startswith("/") else href

            data["탭"].append(cate_a.get_text(strip=True))
            data["제목"].append(title_a.get_text(" ", strip=True))
            data["글쓴이"].append(author_a.get_text(strip=True))
            data["날짜"].append(normalize_date(time_td.get_text(strip=True)))
            data["조회"].append(int(views.replace(",", "")) if views else 0)
            data["추천"].append(int(votes.replace(",", "")) if votes else 0)
            data["post_url"].append(post_url)
            data["source"].append(source_name)

            page_added += 1


        print(f"[{source_name}] {page}페이지 완료 / 이번 페이지 {page_added}개 / 누적 {len(data['post_url'])}개")

        if page_added == 0:
            print(f"[{source_name}] {page}페이지: page_added=0 → 중단")
            break

        time.sleep(sleep_sec)

    return pd.DataFrame(data)


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [5]:
# 검색키원드 "하이닉스" 인기글

url_base_hynix = "https://www.fmkorea.com/search.php?mid=stock&sort_index=pop&order_type=desc&listStyle=list&search_target=title_content&search_keyword=%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&page={}"

# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hynix = requests.Session()
session_hynix.headers.update(headers)  # 세션별 쿠키/헤더 유지

start_page = 1
end_page = 35

# 크롤링 실행
df_hynix = crawl_one(
    session_hynix,
    url_base_hynix,
    "하이닉스",
    start_page=start_page,
    end_page=end_page
)

df_hynix.tail()

[하이닉스] 1페이지 완료 / 이번 페이지 20개 / 누적 20개
[하이닉스] 2페이지 완료 / 이번 페이지 20개 / 누적 40개
[하이닉스] 3페이지 완료 / 이번 페이지 20개 / 누적 60개
[하이닉스] 4페이지 완료 / 이번 페이지 20개 / 누적 80개
[하이닉스] 5페이지 완료 / 이번 페이지 20개 / 누적 100개
[하이닉스] 6페이지 완료 / 이번 페이지 20개 / 누적 120개
[하이닉스] 7페이지 완료 / 이번 페이지 20개 / 누적 140개
[하이닉스] 8페이지 완료 / 이번 페이지 20개 / 누적 160개
[하이닉스] 9페이지 완료 / 이번 페이지 20개 / 누적 180개
[하이닉스] 10페이지 완료 / 이번 페이지 20개 / 누적 200개
[하이닉스] 11페이지 완료 / 이번 페이지 20개 / 누적 220개
[하이닉스] 12페이지 완료 / 이번 페이지 20개 / 누적 240개
[하이닉스] 13페이지 완료 / 이번 페이지 20개 / 누적 260개
[하이닉스] 14페이지 완료 / 이번 페이지 20개 / 누적 280개
[하이닉스] 15페이지 완료 / 이번 페이지 20개 / 누적 300개
[하이닉스] 16페이지 완료 / 이번 페이지 20개 / 누적 320개
[하이닉스] 17페이지 완료 / 이번 페이지 20개 / 누적 340개
[하이닉스] 18페이지 완료 / 이번 페이지 20개 / 누적 360개
[하이닉스] 19페이지 완료 / 이번 페이지 20개 / 누적 380개
[하이닉스] 20페이지 완료 / 이번 페이지 20개 / 누적 400개
[하이닉스] 21페이지 완료 / 이번 페이지 20개 / 누적 420개
[하이닉스] 22페이지 완료 / 이번 페이지 20개 / 누적 440개
[하이닉스] 23페이지 완료 / 이번 페이지 20개 / 누적 460개
[하이닉스] 24페이지 완료 / 이번 페이지 20개 / 누적 480개
[하이닉스] 25페이지 완료 / 이번 페이지 20개 / 누적 500개
[하이닉스] 26페이지 완료 / 이번 페이지 20개 / 누적 520개

,탭,제목,글쓴이,날짜,조회,추천,post_url,source
695,국내주식,친구들랑 국장하면 뭐 모을거냐하고 싸웠는데 ㅋㅋ,운정남,2025-01-06,12629,64,https://www.fmkorea.com/index.php?mid=stock&so...,하이닉스
696,국내주식,아들~ 요새 SK?닉스가 유명하다매~?,호날두.,2025-01-06,9618,83,https://www.fmkorea.com/index.php?mid=stock&so...,하이닉스
697,국내주식,오늘 같은 날 포스코 2억 7천 손절했습니다,하장루프,2025-01-06,16728,103,https://www.fmkorea.com/index.php?mid=stock&so...,하이닉스
698,국내주식,하이닉스 많이 올라가서 좋습네다,TESLA_,2025-01-06,9117,60,https://www.fmkorea.com/index.php?mid=stock&so...,하이닉스
699,해외주식,Today's Point 했제와 그랬제,손흥민,2025-01-06,12870,188,https://www.fmkorea.com/index.php?mid=stock&so...,하이닉스


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%EC%82%BC%EC%A0%84&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [7]:
# 검색키워드 "하닉"

url_base_hanic = "https://www.fmkorea.com/search.php?mid=stock&sort_index=pop&order_type=desc&listStyle=list&search_target=title_content&search_keyword=%ED%95%98%EB%8B%89&page={}"

# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hanic = requests.Session()
session_hanic.headers.update(headers)  # 세션별 쿠키/헤더 유지

start_page = 1
end_page = 20

# 크롤링 실행
df_hanic = crawl_one(
    session_hanic,
    url_base_hanic,
    "하닉",
    start_page=start_page,
    end_page=end_page
)

df_hanic.tail()


[하닉] 1페이지 완료 / 이번 페이지 20개 / 누적 20개
[하닉] 2페이지 완료 / 이번 페이지 20개 / 누적 40개
[하닉] 3페이지 완료 / 이번 페이지 20개 / 누적 60개
[하닉] 4페이지 완료 / 이번 페이지 20개 / 누적 80개
[하닉] 5페이지 완료 / 이번 페이지 20개 / 누적 100개
[하닉] 6페이지 완료 / 이번 페이지 20개 / 누적 120개
[하닉] 7페이지 완료 / 이번 페이지 20개 / 누적 140개
[하닉] 8페이지 완료 / 이번 페이지 20개 / 누적 160개
[하닉] 9페이지 완료 / 이번 페이지 20개 / 누적 180개
[하닉] 10페이지 완료 / 이번 페이지 20개 / 누적 200개
[하닉] 11페이지 완료 / 이번 페이지 20개 / 누적 220개
[하닉] 12페이지 완료 / 이번 페이지 20개 / 누적 240개
[하닉] 13페이지 완료 / 이번 페이지 20개 / 누적 260개
[하닉] 14페이지 완료 / 이번 페이지 20개 / 누적 280개
[하닉] 15페이지 완료 / 이번 페이지 20개 / 누적 300개
[하닉] 16페이지 완료 / 이번 페이지 20개 / 누적 320개
[하닉] 17페이지 완료 / 이번 페이지 20개 / 누적 340개
[하닉] 18페이지 완료 / 이번 페이지 20개 / 누적 360개
[하닉] 19페이지 완료 / 이번 페이지 20개 / 누적 380개
[하닉] 20페이지 완료 / 이번 페이지 20개 / 누적 400개


,탭,제목,글쓴이,날짜,조회,추천,post_url,source
395,국내주식,본인 반도체 관련회사 다니는데 삼성은 앞으로도 별 가망 없어보임 ㅇㅇ,마늘알리오올리오,2024-12-21,13319,57,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
396,국내주식,아까 삼전 HBM 납품실패 기사 블룸버그가 아니고 트렌드포스에 뜨긴 함,도조,2024-12-21,12816,37,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
397,해외주식,국장에 물린 42억을 어떻게 손절하면 좋을지 모르겠습니다,하장루프,2024-12-09,21803,94,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
398,국내주식,13억좌 오랜만에 수익인증 합니다,Fg야요,2024-12-05,8133,77,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
399,국내주식,오늘 국장이 국장 한 게 너무 짜증났지만 그래도 고찰해봄 (feat 경기순환사이클),이붕불,2024-11-30,8960,62,https://www.fmkorea.com/index.php?mid=stock&so...,하닉


csv 파일 생성

In [8]:
# 두 데이터프레임 합치기

df_all = pd.concat([df_hynix, df_hanic], ignore_index=True) 
print(f"합치기 전: {len(df_all):,}")

# 중복 제거: URL 기준
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True) 
print(f"URL 중복 제거 후: {len(df_all):,}")

# 날짜 내림차순 정렬
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

df_all.to_csv("1.csv", index=False, encoding="utf-8-sig")
df_all.tail()

합치기 전: 1,100
URL 중복 제거 후: 1,100


,탭,제목,글쓴이,날짜,조회,추천,post_url,source
1095,국내주식,본인 반도체 관련회사 다니는데 삼성은 앞으로도 별 가망 없어보임 ㅇㅇ,마늘알리오올리오,2024-12-21,13319,57,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
1096,국내주식,아까 삼전 HBM 납품실패 기사 블룸버그가 아니고 트렌드포스에 뜨긴 함,도조,2024-12-21,12816,37,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
1097,해외주식,국장에 물린 42억을 어떻게 손절하면 좋을지 모르겠습니다,하장루프,2024-12-09,21803,94,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
1098,국내주식,13억좌 오랜만에 수익인증 합니다,Fg야요,2024-12-05,8133,77,https://www.fmkorea.com/index.php?mid=stock&so...,하닉
1099,국내주식,오늘 국장이 국장 한 게 너무 짜증났지만 그래도 고찰해봄 (feat 경기순환사이클),이붕불,2024-11-30,8960,62,https://www.fmkorea.com/index.php?mid=stock&so...,하닉


100페이지씩 나누어진 데이터를 하나의 csv로 결합

In [9]:
import pandas as pd

# 1) 합칠 CSV 파일들 (현재 폴더의 csv 전부면 이렇게)
files = ["1.csv"]

# 2) 모두 읽어서 합치기
dfs = [pd.read_csv(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

print("합치기 전:", len(df_all))

# 3) 중복 제거 (post_url 기준이 가장 확실)
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True)  # [web:60]
print("중복 제거 후:", len(df_all))

# 4) 날짜 내림차순 정렬(안전하게 datetime 변환 후 정렬)
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")  # [web:102]
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

# 5) 최종 저장
df_all.to_csv("fm_hynix_pop.csv", index=False, encoding="utf-8-sig")

합치기 전: 1100
중복 제거 후: 1100


## 추출한 게시글 상세정보 (본문내용, 댓글내용) 추출함수 정의

In [10]:
import re
from bs4 import BeautifulSoup as bs
from datetime import datetime
import json

def normalize_date(date_str: str) -> str:
    today = datetime.now().strftime("%Y-%m-%d")
    s = (date_str or "").strip()

    # 예: "11:07" 같은 케이스(인기글에서 시간만 주는 경우) -> 오늘 날짜로
    if re.match(r"^\d{1,2}:\d{2}$", s):
        return today

    # 예: "2026.01.16 11:07" 또는 "2026.01.16" -> "2026-01-16"
    if re.match(r"^\d{4}\.\d{2}\.\d{2}", s):
        s = s.split()[0]          # 시간 잘라내기
        return s.replace(".", "-")

    return s

def parse_int(text):
    # "조회 수 15,720" 같은 문자열에서 숫자만 뽑아 int로 변환
    if text is None:
        return 0
    t = re.sub(r"[^\d]", "", text)
    return int(t) if t else 0

def parse_post_detail(session, post_url):
    
    r = session.get(post_url, timeout=20)
    r.raise_for_status()
    soup = bs(r.text, "lxml")

    # 제목
    title = ""
    title_el = soup.select_one("#bd_capture h1.np_18px span.np_18px_span")
    if title_el:
        title = title_el.get_text(" ", strip=True)

    # 날짜
    date = ""
    date_el = soup.select_one("#bd_capture .top_area .date.m_no")
    if date_el:
        date = normalize_date(date_el.get_text(strip=True))

    # 조회/추천/댓글 수
    views = votes = comment_cnt = 0
    views_b = soup.select_one("#bd_capture .btm_area .side.fr span:nth-of-type(1) b")
    votes_b = soup.select_one("#bd_capture .btm_area .side.fr span:nth-of-type(2) b")
    cmt_b   = soup.select_one("#bd_capture .btm_area .side.fr span:nth-of-type(3) b")
    views = parse_int(views_b.get_text(strip=True) if views_b else None)
    votes = parse_int(votes_b.get_text(strip=True) if votes_b else None)
    comment_cnt = parse_int(cmt_b.get_text(strip=True) if cmt_b else None)

    # 본문 
    content = ""
    content_el = soup.select_one("#bd_capture .rd_body article .xe_content")
    if content_el:
        content = content_el.get_text("\n", strip=True)

    # 댓글 목록
    comments = []
    seen=set()  # (nickname, comment) 중복 체크용
    
    for li in soup.select(".fdb_lst_wrp #cmtPosition ul.fdb_lst_ul > li.fdb_itm.clear"):

        # 닉네임: meta 안 a.member_plate 텍스트
        nick = ""
        nick_el = li.select_one("div.meta a.member_plate")
        if nick_el:
            nick = nick_el.get_text(strip=True)

        # 댓글 내용: comment-content 안 xe_content
        c_text = ""
        text_el = li.select_one(".comment-content .xe_content")
        if text_el:
            c_text = text_el.get_text("\n", strip=True)

        # 댓글 추천수: span.voted_count (없으면 0)
        like = 0
        like_el = li.select_one(".voted_count")
        if like_el:
            like = parse_int(like_el.get_text(strip=True))
        
        # 닉+내용 완전 동일 중복 제거
        key = (nick, c_text)
        if key in seen:
            continue
        seen.add(key)
        
        comments.append({
            "nickname": nick,
            "comment": c_text,
            "like": like
        })
        
    return {
        "post_url": post_url,
        "title": title,
        "date": date,
        "views": views,
        "votes": votes,
        "comment_count": comment_cnt,
        "content": content,
        "comments": comments
    }

def append_jsonl(path, obj):
    # JSONL(JSON Lines) 파일에 "한 줄 = JSON 객체 1개" 형태로 누적 저장하는 함수
    # - mode="a": 기존 파일 뒤에 계속 추가(append)
    # - ensure_ascii=False: 한글을 \uXXXX 이스케이프가 아니라 실제 한글로 저장
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")  


# JSONL 파일은 사람이 보기엔 1줄이 너무 길어서 불편하니,
# JSONL 전체를 읽어서 리스트로 만든 후 pretty JSON(들여쓰기/줄바꿈)으로 저장하는 함수
def export_pretty_json(jsonl_path, pretty_json_path):
    rows = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))  # jsonl 한 줄을 dict로

    with open(pretty_json_path, "w", encoding="utf-8") as f:
        json.dump(rows, f, ensure_ascii=False, indent=2)  # 줄바꿈/들여쓰기


## 실제 추출

In [22]:
import pandas as pd
import requests

df = pd.read_csv("fm_hynix_pop.csv")
urls = df["post_url"].dropna().unique()

session = requests.Session()

# 한번 요청할때 100페이지 이상은 430에러 발생해서 100페이지씩 나누어서 실행
BATCH_SIZE = 100
START = 1000  # <- 재실행할 때 0, 100, 200 ... 으로 바꿔가며 실행

end = min(START + BATCH_SIZE, len(urls))

for i in range(START, end):
    raw_url = str(urls[i])

    obj = parse_post_detail(session, raw_url)
    append_jsonl("fmkorea_hynix_hot_posts.jsonl", obj)

print(f"완료: {START} ~ {end-1} / 전체 {len(urls)}")

#jsonl 파일을 pretty JSON으로 변환
export_pretty_json("fmkorea_hynix_hot_posts.jsonl", "fmkorea_hynix_hot_posts_pretty.json")


완료: 1000 ~ 1099 / 전체 1100
